<a href="https://colab.research.google.com/github/menonajayki/SSL-Context-Encoding/blob/main/SSL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The command is to remove if the directory exist, this ensures clean and consistent starting point before cloning

In [4]:
!if [ -d cs294-158-ssl ]; then rm -Rf cs294-158-ssl; fi

Clone and pip the directory

In [5]:
!git clone https://github.com/wilson1yan/cs294-158-ssl
!pip install cs294-158-ssl/

Cloning into 'cs294-158-ssl'...
remote: Enumerating objects: 611, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 611 (delta 115), reused 119 (delta 53), pack-reused 422
Receiving objects: 100% (611/611), 3.11 MiB | 16.79 MiB/s, done.
Resolving deltas: 100% (424/424), done.
Processing ./cs294-158-ssl
  Preparing metadata (setup.py) ... done
  Created wheel for deepul-helper: filename=deepul_helper-0.1.0-py3-none-any.whl size=25298 sha256=aa0870077d185c1a5cdb3a9f5d499088381ca58b155d63a04b1d24a95dc16b7e
  Stored in directory: /root/.cache/pip/wheels/e3/df/97/281e32d717214451862e0e558746de423b9ef530b7376421e2
Successfully built deepul-helper


Change working directory

In [12]:
import os
os.chdir('cs294-158-ssl') # use .. to go  back in wd

In [20]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /content/cs294-158-ssl


Use gdown to take the data set from google drive

In [21]:
!pip install gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1_3pQdiYZCI-Jybdjj_54OQZmclbddNBk', 'data.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1_3pQdiYZCI-Jybdjj_54OQZmclbddNBk
From (redirected): https://drive.google.com/uc?id=1_3pQdiYZCI-Jybdjj_54OQZmclbddNBk&confirm=t&uuid=64d69134-fe55-4c53-899a-248ba9890264
To: /content/cs294-158-ssl/data.zip
100%|██████████| 4.23G/4.23G [00:41<00:00, 102MB/s]


'data.zip'

In [22]:
!unzip -qq data.zip

In [24]:
!rm data.zip

In [25]:
!ls

build			   environment.yml  setup.py
create_imagenet_subset.py  palette.pkl	    train_segmentation.py
data			   README.md	    train_self_supervised_task.py
deepul_helper		   run
deepul_helper.egg-info	   sample_images


In [26]:
gdown.download("https://drive.google.com/uc?id=1GPQoG4r_zLyOzYrY-YUTl-6EUkCsb99J", "results.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1GPQoG4r_zLyOzYrY-YUTl-6EUkCsb99J
From (redirected): https://drive.google.com/uc?id=1GPQoG4r_zLyOzYrY-YUTl-6EUkCsb99J&confirm=t&uuid=2c0af101-3a12-45a5-823b-0635eb313b3a
To: /content/cs294-158-ssl/results.zip
100%|██████████| 2.64G/2.64G [00:35<00:00, 73.6MB/s]


'results.zip'

In [27]:
!unzip results.zip

Archive:  results.zip
   creating: results/
   creating: results/cifar10_context_encoder/
  inflating: results/cifar10_context_encoder/checkpoint.pth.tar  
  inflating: results/cifar10_context_encoder/model_best.pth.tar  
   creating: results/cifar10_rotation/
  inflating: results/cifar10_rotation/checkpoint.pth.tar  
  inflating: results/cifar10_rotation/model_best.pth.tar  
   creating: results/cifar10_simclr/
  inflating: results/cifar10_simclr/checkpoint.pth.tar  
  inflating: results/cifar10_simclr/model_best.pth.tar  
   creating: results/imagenet100_simclr/
  inflating: results/imagenet100_simclr/checkpoint.pth.tar  
  inflating: results/imagenet100_simclr/model_best.pth.tar  
   creating: results/imagenet100_simclr/segmentation/
  inflating: results/imagenet100_simclr/segmentation/epoch0.png  
  inflating: results/imagenet100_simclr/segmentation/epoch10.png  
  inflating: results/imagenet100_simclr/segmentation/epoch100.png  
  inflating: results/imagenet100_simclr/segmentation

In [28]:
!rm results.zip

Comes from the cs294-158-ssl

In [36]:
from deepul_helper.demos import load_model_and_data, evaluate_accuracy, display_nearest_neighbors, show_context_encoder_inpainting

Line magic function issue

In [37]:
% matplotlib inline

UsageError: Line magic function `%` not found.
